In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('/project/project_462000451/enchanted-surrogates/submodules/static_sparse_grid_approximations')

sys.path.append('/project/project_462000451/enchanted-surrogates/submodules/tokamak_samplers')

In [2]:
from dask_jobqueue import SLURMCluster
job_script_prologue = [
    'export PATH="/project/project_462000451/enchanted_container_lumi3/bin:$PATH"',
    'cd /project/project_462000451/enchanted-surrogates/',
    'export PYTHONPATH=$PYTHONPATH:/project/project_462000451/enchanted-surrogates/src' # NB: to use the enchanted-surrogate library
]

cluster = SLURMCluster(
    # n_workers = 2,
    queue='standard',
    account="project_462000451",
    cores=128, #Total number of cores spread evenly among the workers
    processes=2,
    memory="200GB", #Memory split betwen the workers
    walltime = "04:00:00",
    interface= "nmn0",
    # processes = 2,
    # death_timeout=2,
    job_script_prologue=job_script_prologue
)
n_jobs = 3
cluster.scale(n_jobs)
from dask.distributed import Client
client = Client(cluster)
num_workers = len(client.scheduler_info()["workers"])
print('NUMBER OF WORKERS',num_workers)
print(client)
print('-'*100,'CLUSTER JOB SCRIPT\n',cluster.job_script(),'-'*100)
print(cluster)

/scratch/project_462000451/daniel/daniel_sprint/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45879 instead
  warnings.warn(


NUMBER OF WORKERS 0
<Client: 'tcp://10.252.1.61:43719' processes=0 threads=0, memory=0 B>
---------------------------------------------------------------------------------------------------- CLUSTER JOB SCRIPT
 #!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p standard
#SBATCH -A project_462000451
#SBATCH -n 1
#SBATCH --cpus-per-task=128
#SBATCH --mem=187G
#SBATCH -t 04:00:00
export PATH="/project/project_462000451/enchanted_container_lumi3/bin:$PATH"
cd /project/project_462000451/enchanted-surrogates/
export PYTHONPATH=$PYTHONPATH:/project/project_462000451/enchanted-surrogates/src
/scratch/project_462000451/daniel/daniel_sprint/bin/python -m distributed.cli.dask_worker tcp://10.252.1.61:43719 --name dummy-name --nthreads 64 --memory-limit 93.13GiB --nworkers 2 --nanny --death-timeout 60 --interface nmn0
 ----------------------------------------------------------------------------------------------------
SLURMCluster(c9d81831, 'tcp://10.252.1.61:43719', workers=0, threads=0, memo

In [4]:
client

<Client: 'tcp://10.252.1.61:43719' processes=4 threads=256, memory=372.52 GiB>

In [14]:
import os
import subprocess
from dask.distributed import Client, as_completed
def run_command(commands):
    command = '\n'.join(commands)
    # Run the commands
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    # # Print the output
    # print(result.stdout)
    # print(result.stderr)
    return result.stdout, result.stderr

futures = []
run_base_dir = '/project/project_462000451/gene'
for job_i in range(n_jobs):
    run_dir = os.path.join(run_base_dir, f'dask_{job_i}') 
    commands = [f'cd {run_dir}',
                 'export MEMORY_PER_CORE=1800',
                 'echo 1',
                 'export OMP_NUM_THREADS=1',
                 'echo 2',
                    #do not use file locking for hdf5
                 'export HDF5_USE_FILE_LOCKING=FALSE',
                 'echo 3',
                 'set -x',
                    # run GENE
                    # srun -l -K -n $SLURM_NTASKS ./gene_lumi_csc

                    # run scanscript
                 'echo 4',
                 "./scanscript --np 128 --mps 4 --syscall='srun -l -K -n 128 ./gene_lumi_csc",
                 'echo 5',
                 "set +x"
                ]
    
    futures.append(client.submit(run_command, commands))

results = []
seq = as_completed(futures)
for future in seq:        
    results.append(future.result())
    
for res in results:
    print('-'*100,'\n',res)
    

---------------------------------------------------------------------------------------------------- 
 ('1\n2\n3\n4\n', "+ echo 4\n/bin/sh: -c: line 9: unexpected EOF while looking for matching `''\n/bin/sh: -c: line 12: syntax error: unexpected end of file\n")
---------------------------------------------------------------------------------------------------- 
 ('1\n2\n3\n4\n', "+ echo 4\n/bin/sh: -c: line 9: unexpected EOF while looking for matching `''\n/bin/sh: -c: line 12: syntax error: unexpected end of file\n")
---------------------------------------------------------------------------------------------------- 
 ('1\n2\n3\n4\n', "+ echo 4\n/bin/sh: -c: line 9: unexpected EOF while looking for matching `''\n/bin/sh: -c: line 12: syntax error: unexpected end of file\n")
